In [ ]:
# Import steering angle
import numpy as np
import csv
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import cv2
from io import BytesIO
import scipy.misc
import pandas as pd  

lines = []
with open('driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
images = []
measurements = []
for line in lines:
    for i in range(3):
        measurement = float(line[3])
        correction = 0.21
        source_path = line[i]
        image = cv2.imread(source_path)
        ## Added on 2-12-17
        image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        rows, cols, _ = image_hsv.shape
        transRange = 100
        numPixels = 10
        valPixels = 0.4
        transX = transRange * np.random.uniform() - transRange/2
        steeringAngle = measurement + transX/transRange * 2 * valPixels
        transY = numPixels * np.random.uniform() - numPixels/2
        transMat = np.float32([[1,0, transX], [0,1, transY]])
        image_transformed = cv2.warpAffine(image_hsv, transMat, (cols, rows))
        ###
        images.append(image_hsv)
        images.append(image_transformed)
        
        if i == 0:
            measurements.append(measurement)
            measurements.append(steeringAngle)
            
        if i == 1:
            measurements.append(measurement+correction)
            measurements.append(steeringAngle+correction)
            
        if i == 2:
            measurements.append(measurement-correction)
            measurements.append(steeringAngle-correction)
    
    
print(len(images))
print(len(measurements))

augmented_images = []
augmented_measurements = []
for image, measurement in zip(images,measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    flipped_image = cv2.flip(image,1)
    flipped_measurement = float(measurement) * -1.0
    augmented_images.append(flipped_image)
    augmented_measurements.append(flipped_measurement)

print(len(augmented_images))
print(len(augmented_measurements))
print('Images loaded')


# TODO: Load the feature data to the variable X_train
X_train = np.array(augmented_images)

# TODO: Load the label data to the variable y_train
y_train = np.array(augmented_measurements)

print(X_train.shape)
print(y_train.shape)

print('Data variables sett')


# TODO: Build a model
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, Flatten, Lambda
from keras.layers.convolutional import Convolution2D, Cropping2D
from keras.layers.pooling import MaxPooling2D

pool_size = [2,2]
input_shape = [160, 320, 3]
batch_size = 128
nb_epoch = 7

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=input_shape))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Convolution2D(24,5,5, subsample=pool_size, activation='tanh'))
model.add(Convolution2D(36,5,5, subsample=pool_size, activation='tanh'))
model.add(Convolution2D(48,5,5, subsample=pool_size, activation='tanh'))
model.add(Convolution2D(64,3,3, activation='tanh'))
model.add(Convolution2D(64,3,3, activation='tanh'))
model.add(Flatten())
model.add(Dropout(0.25)) #ADDED
model.add(Dense(100))
model.add(Activation('tanh')) #ADDED
model.add(Dropout(0.25)) #ADDED
model.add(Dense(50))
model.add(Activation('tanh')) #ADDED
model.add(Dense(10))
model.add(Activation('tanh')) #ADDED
model.add(Dense(1))

model.compile('adam', loss='mse')

model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch,verbose=1, validation_split=0.2, shuffle=True) 
#model.fit(X_train, y_train, nb_epoch=nb_epoch, validation_split=0.2, shuffle=True) 


model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')
model.summary()




print('done')

50430
50430
100860
100860
Images loaded
(100860, 160, 320, 3)
(100860,)
Data variables sett
Train on 80688 samples, validate on 20172 samples
Epoch 1/7
80688/80688 [==============================] - 1190s - loss: 0.0474 - val_loss: 0.0383